In [7]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.initializers import GlorotUniform

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
# Step 2: Building the Model
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(128, activation='relu', kernel_initializer=GlorotUniform()))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_initializer=GlorotUniform()))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

is_classification_model = model.layers[-1].activation == 'softmax'

# Rest of the code remains the same
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Step 4: Integrating TensorBoard
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

# Step 5: Baseline with Logistic Regression
logreg = LogisticRegression()
logreg.fit(x_train.reshape(-1, 28*28), y_train)
y_pred = logreg.predict(x_test.reshape(-1, 28*28))
baseline_accuracy = accuracy_score(y_test, y_pred)

# Training the Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[early_stopping, tensorboard_callback], epochs=10)

# Evaluating the Model
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Printing the Results
print("Baseline Accuracy:", baseline_accuracy)
print("Test Accuracy:", test_accuracy)

/home/utilisateur/Bureau/Deep_learning_&_NLP/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8563 - loss: 0.4024 - val_accuracy: 0.8617 - val_loss: 0.3781
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8567 - loss: 0.3928 - val_accuracy: 0.8628 - val_loss: 0.3801
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8644 - loss: 0.3781 - val_accuracy: 0.8634 - val_loss: 0.3699
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8618 - loss: 0.3800 - val_accuracy: 0.8643 - val_loss: 0.3854
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8656 - loss: 0.3670 - val_accuracy: 0.8747 - val_loss: 0.3909
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8670 - loss: 0.3678 - val_accuracy: 0.8574 - val_loss: 0.4939
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8700 - loss: 0.3563 - val_accuracy: 0.8682 - val_loss: 0.4474
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8722 - loss: 0.3510 - 